In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder \
    .appName("Neo4j Data Loading") \
    .config("spark.neo4j.bolt.url", "bolt://neo4j:7687") \
    .config("spark.neo4j.bolt.user", "neo4j") \
    .config("spark.neo4j.bolt.password", "password") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/23 13:49:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/11/23 13:49:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
pg_url = "jdbc:postgresql://postgres:5432/bober_db"
pg_properties = {"user": "bober", "password": "bober", "driver": "org.postgresql.Driver"}
df = spark.read.jdbc(url=pg_url, table="mock_data", properties=pg_properties)

# Загружаем все таблицы звезды
fact = spark.read.jdbc(url=pg_url, table="fact_sales", properties=pg_properties)
dim_product = spark.read.jdbc(url=pg_url, table="dim_product", properties=pg_properties)
dim_customer = spark.read.jdbc(url=pg_url, table="dim_customer", properties=pg_properties)
dim_store = spark.read.jdbc(url=pg_url, table="dim_store", properties=pg_properties)
dim_supplier = spark.read.jdbc(url=pg_url, table="dim_supplier", properties=pg_properties)
dim_date = spark.read.jdbc(url=pg_url, table="dim_date", properties=pg_properties)
dim_date.head(1)

[Row(full_date=datetime.date(2021, 1, 1), date_id=1, year=2021, month=1, day=1, quarter=1)]

In [8]:
def convert_decimal_to_double(df):
    """Конвертирует все DecimalType колонки в DoubleType для совместимости с Neo4j"""
    for field in df.schema.fields:
        if isinstance(field.dataType, DecimalType):
            df = df.withColumn(field.name, col(field.name).cast(DoubleType()))
    return df

def convert_numeric_types(df):
    """Конвертирует все числовые типы для совместимости с Neo4j"""
    for field in df.schema.fields:
        # Конвертируем DecimalType в DoubleType
        if isinstance(field.dataType, DecimalType):
            df = df.withColumn(field.name, col(field.name).cast(DoubleType()))
        # Конвертируем небольшие целые числа в IntegerType
        elif isinstance(field.dataType, (ByteType, ShortType)):
            df = df.withColumn(field.name, col(field.name).cast(IntegerType()))
        # Конвертируем большие целые числа в LongType
        elif isinstance(field.dataType, LongType):
            # Оставляем как есть, Neo4j поддерживает Long
            pass
    return df

# 1. Витрина продаж по продуктам
def create_product_sales_mart():
    product_sales = fact \
        .join(dim_product, "product_id") \
        .groupBy("product_id", "name", "category") \
        .agg(
            sum("sale_quantity").alias("total_quantity"),
            sum("sale_total_price").alias("total_revenue"),
            avg("rating").alias("avg_rating"),
            sum("reviews").alias("total_reviews")
        ) \
        .orderBy(desc("total_quantity")) \
        .limit(10)
    
    return convert_numeric_types(product_sales)

# 2. Витрина продаж по клиентам
def create_customer_sales_mart():
    customer_sales = fact \
        .join(dim_customer, "customer_id") \
        .groupBy("customer_id", "first_name", "last_name", "country") \
        .agg(
            sum("sale_total_price").alias("total_spent"),
            count("sale_id").alias("total_orders"),
            avg("sale_total_price").alias("avg_order_value")
        ) \
        .orderBy(desc("total_spent"))
    
    return convert_numeric_types(customer_sales)

# 3. Витрина продаж по времени
def create_time_sales_mart():
    time_sales = fact \
        .join(dim_date, "date_id") \
        .groupBy("year", "month") \
        .agg(
            sum("sale_total_price").alias("monthly_revenue"),
            count("sale_id").alias("monthly_orders"),
            avg("sale_total_price").alias("avg_order_size")
        ) \
        .orderBy("year", "month")
    
    return convert_numeric_types(time_sales)

# 4. Витрина продаж по магазинам
def create_store_sales_mart():
    store_sales = fact \
        .join(dim_store, "store_id") \
        .groupBy("store_id", "store_name", "store_city", "store_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            count("sale_id").alias("total_sales"),
            avg("sale_total_price").alias("avg_sale_value")
        ) \
        .orderBy(desc("total_revenue"))
    
    return convert_numeric_types(store_sales)

# 5. Витрина продаж по поставщикам
def create_supplier_sales_mart():
    supplier_sales = fact \
        .join(dim_supplier, "supplier_id") \
        .join(dim_product, "product_id") \
        .groupBy("supplier_id", "supplier_name", "supplier_country") \
        .agg(
            sum("sale_total_price").alias("total_revenue"),
            avg("price").alias("avg_product_price"),
            count("sale_id").alias("total_sales")
        ) \
        .orderBy(desc("total_revenue"))
    
    return convert_numeric_types(supplier_sales)

# 6. Витрина качества продукции
def create_product_quality_mart():
    product_quality = dim_product \
        .join(fact, "product_id", "left") \
        .groupBy("product_id", "name", "category", "rating", "reviews") \
        .agg(
            sum("sale_quantity").alias("total_sold"),
            sum("sale_total_price").alias("total_revenue")
        ) \
        .filter(col("rating").isNotNull()) \
        .orderBy(desc("rating"))
    
    return convert_numeric_types(product_quality)

# Основной процесс загрузки данных в Neo4j
def load_all_marts_to_neo4j():
    print("Начало загрузки данных в Neo4j...")
    
    # Конвертируем все таблицы для совместимости с Neo4j
    dim_product_converted = convert_numeric_types(dim_product)
    dim_customer_converted = convert_numeric_types(dim_customer)
    dim_store_converted = convert_numeric_types(dim_store)
    dim_supplier_converted = convert_numeric_types(dim_supplier)
    dim_date_converted = convert_numeric_types(dim_date)
    fact_converted = convert_numeric_types(fact)
    
    # 1. Загрузка основных сущностей
    print("Загрузка продуктов...")
    dim_product_converted.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "Product") \
        .option("node.keys", "product_id") \
        .save()
    
    print("✅ Продукты загружены")
    
    print("Загрузка клиентов...")
    dim_customer_converted.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "Customer") \
        .option("node.keys", "customer_id") \
        .save()
    
    print("✅ Клиенты загружены")
    
    print("Загрузка магазинов...")
    dim_store_converted.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "Store") \
        .option("node.keys", "store_id") \
        .save()
    
    print("✅ Магазины загружены")
    
    print("Загрузка поставщиков...")
    dim_supplier_converted.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "Supplier") \
        .option("node.keys", "supplier_id") \
        .save()
    
    print("✅ Поставщики загружены")
    
    print("Загрузка дат...")
    dim_date_converted.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "Date") \
        .option("node.keys", "date_id") \
        .save()
    
    print("✅ Даты загружены")
    
    print("Загрузка продаж...")
    fact_converted.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "Sale") \
        .option("node.keys", "sale_id") \
        .save()
    
    print("✅ Продажи загружены")
    
    # 2. Загрузка витрин как отдельных узлов
    print("Загрузка витрины продуктов...")
    product_mart = create_product_sales_mart()
    product_mart.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "ProductSalesMart") \
        .option("node.keys", "product_id") \
        .save()
    
    print("✅ Витрина продуктов загружена")
    
    print("Загрузка витрины клиентов...")
    customer_mart = create_customer_sales_mart()
    customer_mart.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "CustomerSalesMart") \
        .option("node.keys", "customer_id") \
        .save()
    
    print("✅ Витрина клиентов загружена")
    
    print("Загрузка витрины времени...")
    time_mart = create_time_sales_mart()
    time_mart.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "TimeSalesMart") \
        .option("node.keys", "year,month") \
        .save()
    
    print("✅ Витрина времени загружена")
    
    print("Загрузка витрины магазинов...")
    store_mart = create_store_sales_mart()
    store_mart.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "StoreSalesMart") \
        .option("node.keys", "store_id") \
        .save()
    
    print("✅ Витрина магазинов загружена")
    
    print("Загрузка витрины поставщиков...")
    supplier_mart = create_supplier_sales_mart()
    supplier_mart.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "SupplierSalesMart") \
        .option("node.keys", "supplier_id") \
        .save()
    
    print("✅ Витрина поставщиков загружена")
    
    print("Загрузка витрины качества...")
    quality_mart = create_product_quality_mart()
    quality_mart.write \
        .format("org.neo4j.spark.DataSource") \
        .mode("Overwrite") \
        .option("url", "bolt://neo4j:7687") \
        .option("authentication.basic.username", "neo4j") \
        .option("authentication.basic.password", "password") \
        .option("labels", "ProductQualityMart") \
        .option("node.keys", "product_id") \
        .save()
    
    print("✅ Витрина качества загружена")
    
    print("🎉 Загрузка данных в Neo4j завершена!")


load_all_marts_to_neo4j()

Начало загрузки данных в Neo4j...
Загрузка продуктов...


✅ Продукты загружены
Загрузка клиентов...


✅ Клиенты загружены
Загрузка магазинов...


✅ Магазины загружены
Загрузка поставщиков...


✅ Поставщики загружены
Загрузка дат...
✅ Даты загружены
Загрузка продаж...


✅ Продажи загружены
Загрузка витрины продуктов...
✅ Витрина продуктов загружена
Загрузка витрины клиентов...


✅ Витрина клиентов загружена
Загрузка витрины времени...
✅ Витрина времени загружена
Загрузка витрины магазинов...
✅ Витрина магазинов загружена
Загрузка витрины поставщиков...


✅ Витрина поставщиков загружена
Загрузка витрины качества...


✅ Витрина качества загружена
🎉 Загрузка данных в Neo4j завершена!
